In [1]:
import pandas as pd
import math
from math import log10, sqrt


# Alter this to fit the path that you have used for the excel file
folder_path = r"C:\Users\rhett\OneDrive\Desktop\PETE Senior Project"

# Define the file name using the same name as the excel file. Copy and paste from the excel file name.
file_name = "Senior_Project_Data.xlsx"

# Concatenate the folder path and file name
excel_file_path = folder_path + "\\" + file_name


In [2]:
import pandas as pd

# Function to load and preprocess data from Excel file
def load_data(sheet_name, excel_file_path):
    try:
        # Load Excel file
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, index_col=0)
        
        # Transpose the DataFrame to switch axes
        df = df.transpose()
        
        # Rename the index to Pressure and columns to Temperature
        df.index.name = "Pressure"
        df.columns.name = "Temperature"
        
        # Convert DataFrame to matrix
        matrix = df.to_numpy()
        
        return df, matrix
    
    except FileNotFoundError:
        print(f"File not found for sheet: {sheet_name}. Please check the file path.")
        return None, None
    except Exception as e:
        print(f"An error occurred while loading data for sheet {sheet_name}: {e}")
        return None, None

# Load data for specific heat, density, and viscosity
df_SH, SH = load_data("SH", excel_file_path)
df_DEN, DEN = load_data("DEN", excel_file_path)
df_VIS, VIS = load_data("VIS", excel_file_path)

# Function to check if value exists in matrix
def value_exists_in_matrix(value, matrix):
    return value in matrix.index and value in matrix.columns

# Print the loaded DataFrames and matrices
print("Specific Heat DataFrame:")
print(df_SH)
print("\nSpecific Heat Matrix:")
print(SH)

print("\nDensity DataFrame:")
print(df_DEN)
print("\nDensity Matrix:")
print(DEN)

print("\nViscosity DataFrame:")
print(df_VIS)
print("\nViscosity Matrix:")
print(VIS)

Specific Heat DataFrame:
Temperature       0            200          400          600          800    \
Pressure                                                                      
-40          772.580000  2007.400000  1992.300000  1978.300000  1965.300000   
-30          778.870000  2040.500000  2021.800000  2004.600000  1988.700000   
-20          785.140000  1110.500000  2059.600000  2038.000000  2018.400000   
-10          791.400000  1072.200000  2108.400000  2080.700000  2055.800000   
 0           797.620000  1042.700000  2172.300000  2135.600000  2103.400000   
 10          803.800000  1020.000000  2258.600000  2207.700000  2164.600000   
 20          809.940000  1002.400000  1492.300000  2305.700000  2245.200000   
 30          816.030000   988.710000  1369.900000  2447.000000  2355.700000   
 40          822.070000   978.100000  1287.500000  2671.600000  2517.300000   
 50          828.060000   969.870000  1228.300000  1968.500000  2780.700000   
 60          833.992274   9

In [3]:
#Subroutine to find SH, DEN, and VIS values using input temperature and pressure values
import numpy as np
import pandas as pd
import math

# Define the path to the Excel file


# Function to load and preprocess data from Excel file
def load_data(sheet_name):
    try:
        # Load Excel file
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, index_col=0)
        
        # Transpose the DataFrame to switch axes
        df = df.transpose()
        
        # Rename the index to Pressure and columns to Temperature
        df.index.name = "Pressure"
        df.columns.name = "Temperature"
        
        # Convert DataFrame to matrix
        matrix = df.to_numpy()
        
        return df, matrix
    
    except FileNotFoundError:
        print(f"File not found for sheet: {sheet_name}. Please check the file path.")
        return None, None
    except Exception as e:
        print(f"An error occurred while loading data for sheet {sheet_name}: {e}")
        return None, None

# Load data for specific heat, density, and viscosity
df_SH, SH = load_data("SH")
df_DEN, DEN = load_data("DEN")
df_VIS, VIS = load_data("VIS")

# Function to check if value exists in matrix
def value_exists_in_matrix(value, matrix):
    return value in matrix.index and value in matrix.columns

def properties(TOPC, POPMA, SH_sheet, DEN_sheet, VIS_sheet):
    # Load data for specific heat, density, and viscosity
    df_SH, SH = load_data(SH_sheet)
    if df_SH is None or SH is None:
        print(f"Failed to load data for {SH_sheet}.")
        return None, None, None
    
    df_DEN, DEN = load_data(DEN_sheet)
    if df_DEN is None or DEN is None:
        print(f"Failed to load data for {DEN_sheet}.")
        return None, None, None
    
    df_VIS, VIS = load_data(VIS_sheet)
    if df_VIS is None or VIS is None:
        print(f"Failed to load data for {VIS_sheet}.")
        return None, None, None
    
    # rest of your code

    
    POP = POPMA * 145.04
    TOP = TOPC * 1.8 + 32
    
    # Adjust TOP if it equals -40.0
    if TOP == -40.0:
        TOP = -39.999
    
    # Check temperature range
    if TOP < -40.0 or TOP > 230.0:
        print('TEMPERATURE OUT OF RANGE: 60 F TO 230 F')
        return None, None, None
    
    # Check pressure range
    POPS = POP + 14.7
    if POPS < 1 or POPS > 12000.0:
        print('PRESSURE OUT OF RANGE: -14.7 PSI TO 11985.3 PSI')
        return None, None, None
    
    # Initialize arrays
    P = [200 * (i - 1) for i in range(1, 62)]
    T = [-40.0 + 10 * (i - 1) for i in range(1, 29)]
    
    # Find P_high and P_low
    for i in range(1, 62):
        if P[i] >= POPS:
            Phigh = P[i]
            Plow = P[i-1]
            KP = i
            break

    # Find T_high and T_low
    for i in range(1, 29):
        if T[i] >= TOP:
            Thigh = T[i]
            Tlow = T[i-1]
            KT = i
            break


    # Interpolate Specific Heat
    A = df_SH.iloc[KT-1, KP-1]
    B = df_SH.iloc[KT-1, KP]
    C = df_SH.iloc[KT, KP-1]
    D = df_SH.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    SHOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Viscosity
    A = df_VIS.iloc[KT-1, KP-1]
    B = df_VIS.iloc[KT-1, KP]
    C = df_VIS.iloc[KT, KP-1]
    D = df_VIS.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    VISOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Density
    A = df_DEN.iloc[KT-1, KP-1]
    B = df_DEN.iloc[KT-1, KP]
    C = df_DEN.iloc[KT, KP-1]
    D = df_DEN.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    DENOP = (X + (Y - X) * (POPS - Plow) / 200.0) * 1000

    return SHOP, VISOP, DENOP

# Incorporating the rest of the code

def Temp(D1, D2, D5, D6, V1, V2, V5, V6, C1, C2, C5, C6, T2, T5, TF, U, A, DI, DX, DT):
    DA = (D1 + D2 + D5 + D6) / 4
    VA = (V1 + V2 + V5 + V6) / 4
    CA = (C1 + C2 + C5 + C6) / 4
    XA = (A * DA * VA * CA) / DX
    TA = (A * DA * CA) / DT
    SA = 3.1416 * DI * U
    T = (XA * T5 + TA * T2 + SA * TF) / (XA + TA + SA)
    
    return T

def Velc(D1, D2, D5, D6, V1, V2, V5, DX, DT):
    A = (D5 + D6 - D1 - D2) / (2 * DT)
    B = (V2 * D2 - V1 * D1 - V5 * D5) / (2 * DX)
    V = -(A + B) * 2 * DX / D6
    
    return V

def fricg(VSI, GV, D1SI, D2SI, DDSI):
    V = VSI * 3.281
    D1 = D1SI * 100 / 2.54
    D2 = D2SI * 100 / 2.54
    DD = (DDSI / 1000) * 8.34
    
    if D2 < 0.001:
        A = 757
        B = 21.1
    else:
        A = 928
        B = 25.8
    
    REY = A * DD * V * (D1 - D2) / GV
    GL = GV * V / (1000 * (D1 - D2) ** 2)
    F = frfact(REY, D1, D2)
    GT = F * DD * V ** 2 / (B * (D1 - D2))
    
    if GL < GT:
        GFP = GT
    else:
        GFP = GL
    
    GFPSI = GFP * 0.0226206
    
    return GFPSI

def frfact(REY, D1, D2):
    ED = 0.00025 / (D1 - D2)
    FGI = 0.0056 + 0.5 / REY ** 0.32
    I = 1
    
    while True:
        DEN = 1.14 - 2.0 * math.log10(ED + 9.34 / (REY * math.sqrt(FGI)))
        FF = (1.0 / DEN) ** 2
        DIFF = abs(FGI - FF)
        
        if DIFF <= 1.0E-4:
            break
        
        FGI = (FGI + FF) / 2
        I += 1
        
        if I >= 10:
            print("THE FRICTION FACTOR DOES NOT CONVERGE")
            return None
    
    return FF / 4

# Example usage
TOPC = float(input("Enter temperature value (C): "))
POPMA = float(input("Enter pressure value (MPa): "))

sheet_SH = "SH"
sheet_DEN = "DEN"
sheet_VIS = "VIS"
SH_units = "J/kg-C"
DEN_units = "kg/m^3"
VIS_units = "cp"

SHOP, VISOP, DENOP = properties(TOPC, POPMA, sheet_SH, sheet_DEN, sheet_VIS)
print("Specific Heat:", SHOP, SH_units)
print("Viscosity:", VISOP, VIS_units)
print("Density:", DENOP, DEN_units)


Enter temperature value (C): 24
Enter pressure value (MPa): 80
Specific Heat: 1652.607796 J/kg-C
Viscosity: 0.1688257434 cp
Density: 1102.763454 kg/m^3


In [10]:
# Steady-State and Time Loop Code
import numpy as np
import pandas as pd
import math

#TOPC = float(input("Enter temperature value (C): "))
#POPMA = float(input("Enter pressure value (MPa): "))

# Function to load and preprocess data from Excel file
def load_data(sheet_name):
    try:
        # Load Excel file
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, index_col=0)
        
        # Transpose the DataFrame to switch axes
        df = df.transpose()
        
        # Rename the index to Pressure and columns to Temperature
        df.index.name = "Pressure"
        df.columns.name = "Temperature"
        
        # Convert DataFrame to matrix
        matrix = df.to_numpy()
        
        return df, matrix
    
    except FileNotFoundError:
        print(f"File not found for sheet: {sheet_name}. Please check the file path.")
        return None, None
    except Exception as e:
        print(f"An error occurred while loading data for sheet {sheet_name}: {e}")
        return None, None

# Define the PROPERTIES function with the same signature as in Fortran
def properties(TOPC, POPMA, SH_sheet, DEN_sheet, VIS_sheet):
    # Load data for specific heat, density, and viscosity
    df_SH, SH = load_data(SH_sheet)
    if df_SH is None or SH is None:
        print(f"Failed to load data for {SH_sheet}.")
        return None, None, None
    
    df_DEN, DEN = load_data(DEN_sheet)
    if df_DEN is None or DEN is None:
        print(f"Failed to load data for {DEN_sheet}.")
        return None, None, None
    
    df_VIS, VIS = load_data(VIS_sheet)
    if df_VIS is None or VIS is None:
        print(f"Failed to load data for {VIS_sheet}.")
        return None, None, None
    
    # rest of your code
    
    POP = POPMA * 145.04
    TOP = TOPC * 1.8 + 32
    
    # Adjust TOP if it equals -40.0
    if TOP == -40.0:
        TOP = -39.999
    
    # Check temperature range
    if not (-40.0 <= TOP <= 230.0):
        print('TEMPERATURE OUT OF RANGE: 60 F TO 230 F')
        return None, None, None
    
    # Check pressure range
    POPS = POP + 14.7
    if not (1 <= POPS <= 12000.0):
        print('PRESSURE OUT OF RANGE: -14.7 PSI TO 11985.3 PSI')
        return None, None, None
    
    # Initialize arrays
    P = [200 * (i - 1) for i in range(1, 62)]
    T = [-40.0 + 10 * (i - 1) for i in range(1, 29)]
    
    # Find P_high and P_low
    for i in range(1, 62):
        if P[i] >= POPS:
            Phigh = P[i]
            Plow = P[i-1]
            KP = i
            break

    # Find T_high and T_low
    for i in range(1, 29):
        if T[i] >= TOP:
            Thigh = T[i]
            Tlow = T[i-1]
            KT = i
            break

    # Convert indices to integers
    KP = int(KP)
    KT = int(KT)


    # Interpolate Specific Heat
    A = df_SH.iloc[KT-1, KP-1]
    B = df_SH.iloc[KT-1, KP]
    C = df_SH.iloc[KT, KP-1]
    D = df_SH.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    SHOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Viscosity
    A = df_VIS.iloc[KT-1, KP-1]
    B = df_VIS.iloc[KT-1, KP]
    C = df_VIS.iloc[KT, KP-1]
    D = df_VIS.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    VISOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Density
    A = df_DEN.iloc[KT-1, KP-1]
    B = df_DEN.iloc[KT-1, KP]
    C = df_DEN.iloc[KT, KP-1]
    D = df_DEN.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    DENOP = (X + (Y - X) * (POPS - Plow) / 200.0) * 1000

    return SHOP, VISOP, DENOP

# Incorporating the rest of the code

def Temp(D1, D2, D5, D6, V1, V2, V5, V6, C1, C2, C5, C6, T2, T5, TF, U, A, DI, DX, DT):
    DA = (D1 + D2 + D5 + D6) / 4
    VA = (V1 + V2 + V5 + V6) / 4
    CA = (C1 + C2 + C5 + C6) / 4
    XA = (A * DA * VA * CA) / DX
    TA = (A * DA * CA) / DT
    SA = 3.1416 * DI * U
    T = (XA * T5 + TA * T2 + SA * TF) / (XA + TA + SA)
    
    return T

def Velc(D1, D2, D5, D6, V1, V2, V5, DX, DT):
    A = (D5 + D6 - D1 - D2) / (2 * DT)
    B = (V2 * D2 - V1 * D1 - V5 * D5) / (2 * DX)
    V = -(A + B) * 2 * DX / D6
    
    return V

def fricg(VSI, GV, D1SI, D2SI, DDSI):
    V = VSI * 3.281
    D1 = D1SI * 100 / 2.54
    D2 = D2SI * 100 / 2.54
    DD = (DDSI / 1000) * 8.34
    
    if D2 < 0.001:
        A = 757
        B = 21.1
    else:
        A = 928
        B = 25.8
    
    REY = A * DD * V * (D1 - D2) / GV
    GL = GV * V / (1000 * (D1 - D2) ** 2)
    F = frfact(REY, D1, D2)
    GT = F * DD * V ** 2 / (B * (D1 - D2))
    
    if GL < GT:
        GFP = GT
    else:
        GFP = GL
    
    GFPSI = GFP * 0.0226206
    
    return GFPSI

def frfact(REY, D1, D2):
    ED = 0.00025 / (D1 - D2)
    FGI = 0.0056 + 0.5 / REY ** 0.32
    I = 1
    
    while True:
        DEN = 1.14 - 2.0 * math.log10(ED + 9.34 / (REY * math.sqrt(FGI)))
        FF = (1.0 / DEN) ** 2
        DIFF = abs(FGI - FF)
        
        if DIFF <= 1.0E-4:
            break
        
        FGI = (FGI + FF) / 2
        I += 1
        
        if I >= 10:
            print("THE FRICTION FACTOR DOES NOT CONVERGE")
            return None
    
    return FF / 4


# Parameters definition
PINJ = 80.  # Injection pressure (MPa)
TINJ = -20.  # Injection temperature (C)
TS = 24.  # Formation surface temperature (C)
GG = 2.95 / 100  # Geothermal gradient (C/100m)
WD = 2800  # Well depth (m)
QM = 4.0  # Injection rate (m3/min)
DI = 0.076  # Tubing internal diameter (m)
U = 100.  # Heat transfer coefficient (W/(m2.C))
A = np.pi * DI**2 / 4  # Tubing section area (m2)
VI = QM / (A * 60)  # Velocity (m/s)
DX = 50.  # Element length (m)
N = int(WD / DX)  # Number of elements
DT = 3  # Time step size (s)
TIME = np.zeros(100)  # Time array

# Setting grid depths and temperatures
D = np.zeros(100)
TF = np.zeros(100)
for I in range(N):
    D[I] = DX * I
    TF[I] = TS + GG * D[I]

# Define matrices
P = np.zeros((100, 100))
T = np.zeros((100, 100))
V = np.zeros((100, 100))
SH = np.zeros((100, 100))
DEN = np.zeros((100, 100))
VIS = np.zeros((100, 100))

# Defining the initial conditions
P[0, 0] = PINJ
T[0, 0] = TS
V[0, 0] = VI
_, _, DEN[0, 0] = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
_, VIS[0, 0], _ = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
SH[0 ,0], _, _ = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
# Accumulate viscosity values
#viscosity_values = []

for I in range(N):
    PA = P[0, 0]
    TA = T[0, 0]
    _, _, DEN[I + 1, 0] = properties(TA, PA, "SH", "DEN", "VIS")
    _, VIS[I + 1, 0], _ = properties(TA, PA, "SH", "DEN", "VIS")
    SH[I + 1, 0], _, _ = properties(TA, PA, "SH", "DEN", "VIS")
    DENA = (DEN[I, 0] + DEN[I + 1, 0]) / 2
    VISA = (VIS[I, 0] + VIS[I + 1, 0]) / 2

    # Mass balance equation
    if DEN[I + 1, 0] != 0:
        V[I + 1, 0] = V[I, 0] * DEN[I, 0] / DEN[I + 1, 0]
    else:
        # Handle the case where DEN[I + 1, 0] is zero
        V[I + 1, 0] = 0  # or any other suitable value

    VA = (V[I, 0] + V[I + 1, 0]) / 2

#print("Average Velocity:", VA)
#print("Average Viscosity:", VISA)
#print("Average Density:", DENA)

# Momentun balance equation
for I in range(N):
    GPH = 9.81 * DENA / 1000000.  # Converted from Fortran
    DP = (GPH - fricg(VA, VISA, DI, 0., DENA)) * DX  # Using fricg from Code 2
    
    P[I + 1, 0] = P[I, 0] + DP

# Heat Transfer equation
for I in range(N):
    A1 = A * DEN[I + 1, 0] * V[I + 1, 0] * SH[I + 1, 0] / DX
    B1 = 3.1416 * U * DI
    T[I + 1, 0] = (A1 * T[I, 0] + B1 * TF[I + 1]) / (A1 + B1)

    if abs(P[I + 1, 0] - PA) < 0.1 and abs(T[I + 1, 0] - TA) < 0.1:
        break

    PA = P[I + 1, 0]
    TA = T[I + 1, 0]
   
    #print([I], D[I], P[I, 0], T[I, 0] + 273, TF[I] + 273)  # Print values (remove this line)

# Open files for writing
with open('Resulpm.txt', 'w') as file1, open('Resultm.txt', 'w') as file2:
    # Write values to Resulpm.txt
    for I in range(N):
        file1.write(f'{I} {D[I]} {P[I, 0]} {T[I, 0] + 273} {TF[I] + 273}\n')

    # Write values to Resultm.txt
    for I in range(N):
        file2.write(f'{I} {D[I]} {P[I, 0]} {T[I, 0] + 273} {TF[I] + 273}\n')


#Beginning of Time Loop Code

Time = np.zeros(100)  # Time array
Time[0] = 0  # Initialize the first time step



# Setting grid depths and temperatures


for J in range(1, 91):  
    Time[J] = Time[J - 1] + DT / 60. 
    P[0, J] = PINJ
    T[0, J] = TINJ
    V[0, J] = VI
    _, _, DEN[0, J] = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    _, VIS[0, J], _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    SH[0, J], _, _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")

    for I in range(N):
        PA = P[0, J]  # Note the change here
        TA = T[0, J]  # Note the change here
        _, _, DEN[I + 1, J] = properties(TA, PA, "SH", "DEN", "VIS")
        _, VIS[I + 1, J], _ = properties(TA, PA, "SH", "DEN", "VIS")
        SH[I + 1, J], _, _ = properties(TA, PA, "SH", "DEN", "VIS")
        
        #_, _, DEN[I, J-1] = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        #_, VIS[I, J-1], _ = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        #SH[I, J-1], _, _ = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        
        DENA = (DEN[I, J] + DEN[I + 1, J]) / 2
        VISA = (VIS[I, J] + VIS[I + 1, J]) / 2

        # Mass balance equation
        if DEN[I + 1, J] != 0:
            V[I + 1, J] = V[I, J] * DEN[I, J] / DEN[I + 1, J]
        else:
            # Handle the case where DEN[I + 1, J] is zero
            V[I + 1, J] = 0  # or any other suitable value

        VA = (V[I, J] + V[I + 1, J]) / 2

        #print("Time Step:", Time[J])
        #print("Depth Element:", I)
        #print("Average Velocity:", VA)
        #print("Average Viscosity:", VISA)
        #print("Average Density:", DENA)



        # Momentun balance equation
        GPH = 9.81 * DENA / 1000000.  # Converted from Fortran
        DP = (GPH - fricg(VA, VISA, DI, 0., DENA)) * DX  # Using fricg from Code 2
        P[I + 1, J] = P[I, J] + DP
        #Heat Transfer Equation
        T[I+1, J] = Temp(DEN[I, J-1], DEN[I+1, J-1], DEN[I, J], DEN[I+1, J], V[I, J-1], V[I+1, J-1], V[I, J], V[I+1, J],
                SH[I, J-1], SH[I+1, J-1], SH[I, J], SH[I+1, J], T[I+1, J-1], T[I, J], TF[I+1], U, A, DI, DX, DT)

    
    
    
        if abs(P[I+1, J] - PA) < 0.1 and abs(T[I+1, J] - TA) < 0.1:
            break

            
        #print([I], D[I], P[I, J])

            
        #print([I], D[I], P[I, J])
            
# Writing results to files
with open("Resulpm2.txt", "w") as f1, open("Resultm2.txt", "w") as f2:
    for J in range(100):  # Iterate over the entire time range
        for I in range(N):
            f1.write(f"{Time[J]} {D[I]} {P[I, J]} ")
            f2.write(f"{Time[J]} {D[I]} {T[I, J] + 273.} ")
        f1.write("\n")
        f2.write("\n")

In [45]:
#Time Loop Code
import numpy as np
import pandas as pd
import math

#TOPC = float(input("Enter temperature value (C): "))
#POPMA = float(input("Enter pressure value (MPa): "))


# Define the PROPERTIES function with the same signature as in Fortran
def properties(TOPC, POPMA, SH_sheet, DEN_sheet, VIS_sheet):
    # Load data for specific heat, density, and viscosity
    df_SH, SH = load_data(SH_sheet)
    if df_SH is None or SH is None:
        print(f"Failed to load data for {SH_sheet}.")
        return None, None, None
    
    df_DEN, DEN = load_data(DEN_sheet)
    if df_DEN is None or DEN is None:
        print(f"Failed to load data for {DEN_sheet}.")
        return None, None, None
    
    df_VIS, VIS = load_data(VIS_sheet)
    if df_VIS is None or VIS is None:
        print(f"Failed to load data for {VIS_sheet}.")
        return None, None, None
    
    # rest of your code
    
    POP = POPMA * 145.04
    TOP = TOPC * 1.8 + 32
    
    # Adjust TOP if it equals -40.0
    if TOP == -40.0:
        TOP = -39.999
    
    # Check temperature range
    if not (-40.0 <= TOP <= 230.0):
        print('TEMPERATURE OUT OF RANGE: 60 F TO 230 F')
        return None, None, None
    
    # Check pressure range
    POPS = POP + 14.7
    if not (1 <= POPS <= 12000.0):
        print('PRESSURE OUT OF RANGE: -14.7 PSI TO 11985.3 PSI')
        return None, None, None
    
    # Initialize arrays
    P = [200 * (I - 1) for I in range(1, 62)]
    T = [-40.0 + 10 * (I - 1) for I in range(1, 29)]
    
    # Find P_high and P_low
    for I in range(1, 62):
        if P[I] >= POPS:
            Phigh = P[I]
            Plow = P[I-1]
            KP = I
            break

    # Find T_high and T_low
    for I in range(1, 29):
        if T[I] >= TOP:
            Thigh = T[I]
            Tlow = T[I-1]
            KT = I
            break

    # Convert indices to integers
    KP = int(KP)
    KT = int(KT)


    # Interpolate Specific Heat
    A = df_SH.iloc[KT-1, KP-1]
    B = df_SH.iloc[KT-1, KP]
    C = df_SH.iloc[KT, KP-1]
    D = df_SH.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    SHOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Viscosity
    A = df_VIS.iloc[KT-1, KP-1]
    B = df_VIS.iloc[KT-1, KP]
    C = df_VIS.iloc[KT, KP-1]
    D = df_VIS.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    VISOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Density
    A = df_DEN.iloc[KT-1, KP-1]
    B = df_DEN.iloc[KT-1, KP]
    C = df_DEN.iloc[KT, KP-1]
    D = df_DEN.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    DENOP = (X + (Y - X) * (POPS - Plow) / 200.0) * 1000

    return SHOP, VISOP, DENOP

# Incorporating the rest of the code

def Temp(D1, D2, D5, D6, V1, V2, V5, V6, C1, C2, C5, C6, T2, T5, TF, U, A, DI, DX, DT):
    DA = (D1 + D2 + D5 + D6) / 4
    VA = (V1 + V2 + V5 + V6) / 4
    CA = (C1 + C2 + C5 + C6) / 4
    XA = (A * DA * VA * CA) / DX
    TA = (A * DA * CA) / DT
    SA = 3.1416 * DI * U
    T = (XA * T5 + TA * T2 + SA * TF) / (XA + TA + SA)
    
    return T

def Velc(D1, D2, D5, D6, V1, V2, V5, DX, DT):
    A = (D5 + D6 - D1 - D2) / (2 * DT)
    B = (V2 * D2 - V1 * D1 - V5 * D5) / (2 * DX)
    V = -(A + B) * 2 * DX / D6
    
    return V

def fricg(VSI, GV, D1SI, D2SI, DDSI):
    V = VSI * 3.281
    D1 = D1SI * 100 / 2.54
    D2 = D2SI * 100 / 2.54
    DD = (DDSI / 1000) * 8.34
    
    if D2 < 0.001:
        A = 757
        B = 21.1
    else:
        A = 928
        B = 25.8
    
    #print("Debugging: Values for calculating REY:")
    #print("VSI:", VSI)
    #print("GV:", GV)
    #print("D1SI:", D1SI)
    #print("D2SI:", D2SI)
    #print("DDSI:", DDSI)
    #print("V:", V)
    #print("D1:", D1)
    #print("D2:", D2)
    #print("DD:", DD)
    
    REY = A * DD * V * (D1 - D2) / GV
    
    #print("Debugging: Value of REY:", REY)
    
    GL = GV * V / (1000 * (D1 - D2) ** 2)
    F = frfact(REY, D1, D2)
    GT = F * DD * V ** 2 / (B * (D1 - D2))
    
    if GL < GT:
        GFP = GT
    else:
        GFP = GL
    
    GFPSI = GFP * 0.0226206
    
    return GFPSI


def frfact(REY, D1, D2):
    ED = 0.00025 / (D1 - D2)
    FGI = 0.0056 + 0.5 / REY ** 0.32
    I = 1
    
    while True:
        DEN = 1.14 - 2.0 * math.log10(ED + 9.34 / (REY * math.sqrt(FGI)))
        FF = (1.0 / DEN) ** 2
        DIFF = abs(FGI - FF)
        
        if DIFF <= 1.0E-4:
            break
        
        FGI = (FGI + FF) / 2
        I += 1
        
        if I >= 10:
            print("THE FRICTION FACTOR DOES NOT CONVERGE")
            return None
    
    return FF / 4

# Parameters definition
PINJ = 80.  # Injection pressure (MPa)
TINJ = -20.  # Injection temperature (C)
TS = 24.  # Formation surface temperature (C)
GG = 2.95 / 100  # Geothermal gradient (C/100m)
WD = 2800  # Well depth (m)
QM = 4.0  # Injection rate (m3/min)
DI = 0.076  # Tubing internal diameter (m)
U = 100.  # Heat transfer coefficient (W/(m2.C))
A = np.pi * DI**2 / 4  # Tubing section area (m2)
VI = QM / (A * 60)  # Velocity (m/s)
DX = 50.  # Element length (m)
N = int(WD / DX)  # Number of elements
DT = 3  # Time step size (s)
Time = np.zeros(101)  # Time array
Time[0] = 0  # Initialize the first time step



# Setting grid depths and temperatures
D = np.zeros(101)
TF = np.zeros(101)
for I in range(N + 1):
    D[I] = DX * I
    TF[I] = TS + GG * D[I]

# Define matrices
P = np.zeros((101, 101))
T = np.zeros((101, 101))
V = np.zeros((101, 101))
SH = np.zeros((101, 101))
DEN = np.zeros((101, 101))
VIS = np.zeros((101, 101))

for J in range(2, 101):  
    Time[J] = Time[J - 1] + DT / 60. 
    P[0, J] = PINJ
    T[0, J] = TINJ
    V[0, J] = VI
    _, _, DEN[0, J] = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    _, VIS[0, J], _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    SH[0, J], _, _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")

    for I in range(N):
        PA = P[I, J]  # Note the change here
        TA = T[I, J]  # Note the change here
        _, _, DEN[I + 1, J] = properties(TA, PA, "SH", "DEN", "VIS")
        _, VIS[I + 1, J], _ = properties(TA, PA, "SH", "DEN", "VIS")
        SH[I + 1, J], _, _ = properties(TA, PA, "SH", "DEN", "VIS")
        
        DENA = (DEN[I, J] + DEN[I + 1, J]) / 2
        VISA = (VIS[I, J] + VIS[I + 1, J]) / 2

        # Mass balance equation
        if DEN[I + 1, J] != 0:
            V[I + 1, J] = V[I, J] * DEN[I, J] / DEN[I + 1, J]
        else:
            # Handle the case where DEN[I + 1, J] is zero
            V[I + 1, J] = 0  # or any other suitable value

        VA = (V[I, J] + V[I + 1, J]) / 2

        #print("Time Step:", Time[J])
        #print("Depth Element:", I)
        #print("Average Velocity:", VA)
        #print("Average Viscosity:", VISA)
        #print("Average Density:", DENA)



        # Momentun balance equation
        GPH = 9.81 * DENA / 1000000.  # Converted from Fortran
        DP = (GPH - fricg(VA, VISA, DI, 0., DENA)) * DX  # Using fricg from Code 2
        P[I + 1, J] = P[I, J] + DP
        #Heat Transfer Equation
        T[I+1, J] = Temp(DEN[I, J-1], DEN[I+1, J-1], DEN[I, J], DEN[I+1, J], V[I, J-1], V[I+1, J-1], V[I, J], V[I+1, J],
                SH[I, J-1], SH[I+1, J-1], SH[I, J], SH[I+1, J], T[I+1, J-1], T[I, J], TF[I+1], U, A, DI, DX, DT)

    
    
    
        if abs(P[I+1, J] - PA) < 0.1 and abs(T[I+1, J] - TA) < 0.1:
            break

            
        #print([I], D[I], P[I, J])
            
# Writing results to files
with open("Resulpm.txt", "w") as f1, open("Resultm.txt", "w") as f2:
    for J in range(101):  # Iterate over the entire time range
        for I in range(N + 1):
            f1.write(f"{Time[J]} {D[I]} {P[I, J]} ")
            f2.write(f"{Time[J]} {D[I]} {T[I, J] + 273.} ")
        f1.write("\n")
        f2.write("\n")



In [9]:
# Steady-State Code
import numpy as np
import pandas as pd
import math

#TOPC = float(input("Enter temperature value (C): "))
#POPMA = float(input("Enter pressure value (MPa): "))

# Function to load and preprocess data from Excel file
def load_data(sheet_name):
    try:
        # Load Excel file
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, index_col=0)
        
        # Transpose the DataFrame to switch axes
        df = df.transpose()
        
        # Rename the index to Pressure and columns to Temperature
        df.index.name = "Pressure"
        df.columns.name = "Temperature"
        
        # Convert DataFrame to matrix
        matrix = df.to_numpy()
        
        return df, matrix
    
    except FileNotFoundError:
        print(f"File not found for sheet: {sheet_name}. Please check the file path.")
        return None, None
    except Exception as e:
        print(f"An error occurred while loading data for sheet {sheet_name}: {e}")
        return None, None

# Define the PROPERTIES function with the same signature as in Fortran
def properties(TOPC, POPMA, SH_sheet, DEN_sheet, VIS_sheet):
    # Load data for specific heat, density, and viscosity
    df_SH, SH = load_data(SH_sheet)
    if df_SH is None or SH is None:
        print(f"Failed to load data for {SH_sheet}.")
        return None, None, None
    
    df_DEN, DEN = load_data(DEN_sheet)
    if df_DEN is None or DEN is None:
        print(f"Failed to load data for {DEN_sheet}.")
        return None, None, None
    
    df_VIS, VIS = load_data(VIS_sheet)
    if df_VIS is None or VIS is None:
        print(f"Failed to load data for {VIS_sheet}.")
        return None, None, None
    
    # rest of your code
    
    POP = POPMA * 145.04
    TOP = TOPC * 1.8 + 32
    
    # Adjust TOP if it equals -40.0
    if TOP == -40.0:
        TOP = -39.999
    
    # Check temperature range
    if not (-40.0 <= TOP <= 230.0):
        print('TEMPERATURE OUT OF RANGE: 60 F TO 230 F')
        return None, None, None
    
    # Check pressure range
    POPS = POP + 14.7
    if not (1 <= POPS <= 12000.0):
        print('PRESSURE OUT OF RANGE: -14.7 PSI TO 11985.3 PSI')
        return None, None, None
    
    # Initialize arrays
    P = [200 * (i - 1) for i in range(1, 62)]
    T = [-40.0 + 10 * (i - 1) for i in range(1, 29)]
    
    # Find P_high and P_low
    for i in range(1, 62):
        if P[i] >= POPS:
            Phigh = P[i]
            Plow = P[i-1]
            KP = i
            break

    # Find T_high and T_low
    for i in range(1, 29):
        if T[i] >= TOP:
            Thigh = T[i]
            Tlow = T[i-1]
            KT = i
            break

    # Convert indices to integers
    KP = int(KP)
    KT = int(KT)


    # Interpolate Specific Heat
    A = df_SH.iloc[KT-1, KP-1]
    B = df_SH.iloc[KT-1, KP]
    C = df_SH.iloc[KT, KP-1]
    D = df_SH.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    SHOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Viscosity
    A = df_VIS.iloc[KT-1, KP-1]
    B = df_VIS.iloc[KT-1, KP]
    C = df_VIS.iloc[KT, KP-1]
    D = df_VIS.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    VISOP = X + (Y - X) * (POPS - Plow) / 200.0

    # Interpolate Density
    A = df_DEN.iloc[KT-1, KP-1]
    B = df_DEN.iloc[KT-1, KP]
    C = df_DEN.iloc[KT, KP-1]
    D = df_DEN.iloc[KT, KP]
    X = A + (C - A) * (TOP - Tlow) / 10.0
    Y = B + (D - B) * (TOP - Tlow) / 10.0
    DENOP = (X + (Y - X) * (POPS - Plow) / 200.0) * 1000

    return SHOP, VISOP, DENOP

# Incorporating the rest of the code

def Temp(D1, D2, D5, D6, V1, V2, V5, V6, C1, C2, C5, C6, T2, T5, TF, U, A, DI, DX, DT):
    DA = (D1 + D2 + D5 + D6) / 4
    VA = (V1 + V2 + V5 + V6) / 4
    CA = (C1 + C2 + C5 + C6) / 4
    XA = (A * DA * VA * CA) / DX
    TA = (A * DA * CA) / DT
    SA = 3.1416 * DI * U
    T = (XA * T5 + TA * T2 + SA * TF) / (XA + TA + SA)
    
    return T

def Velc(D1, D2, D5, D6, V1, V2, V5, V6, DX, DT):
    A = (D5 + D6 - D1 - D2) / (2 * DT)
    B = (V2 * D2 - V1 * D1 - V5 * D5) / (2 * DX)
    V = -(A + B) * 2 * DX / D6
    
    return V

def fricg(VSI, GV, D1SI, D2SI, DDSI):
    V = VSI * 3.281
    D1 = D1SI * 100 / 2.54
    D2 = D2SI * 100 / 2.54
    DD = (DDSI / 1000) * 8.34
    
    if D2 < 0.001:
        A = 757
        B = 21.1
    else:
        A = 928
        B = 25.8
    
    REY = A * DD * V * (D1 - D2) / GV
    GL = GV * V / (1000 * (D1 - D2) ** 2)
    F = frfact(REY, D1, D2)
    GT = F * DD * V ** 2 / (B * (D1 - D2))
    
    if GL < GT:
        GFP = GT
    else:
        GFP = GL
    
    GFPSI = GFP * 0.0226206
    
    print(f"REY: {REY}, GL: {GL}, GT: {GT}, F: {F}, GFP: {GFP}, GFPSI: {GFPSI}")
    
    return GFPSI

def frfact(REY, D1, D2):
    ED = 0.00025 / (D1 - D2)
    FGI = 0.0056 + 0.5 / REY ** 0.32
    I = 1
    
    while True:
        DEN = 1.14 - 2.0 * math.log10(ED + 9.34 / (REY * math.sqrt(FGI)))
        FF = (1.0 / DEN) ** 2
        DIFF = abs(FGI - FF)
        
        if DIFF <= 1.0E-4:
            break
        
        FGI = (FGI + FF) / 2
        I += 1
        
        if I >= 10:
            print("THE FRICTION FACTOR DOES NOT CONVERGE")
            return None
    
    return FF / 4


# Parameters definition
PINJ = 80.  # Injection pressure (MPa)
TINJ = -20.  # Injection temperature (C)
TS = 24.  # Formation surface temperature (C)
GG = 2.95 / 100  # Geothermal gradient (C/100m)
WD = 2800  # Well depth (m)
QM = 4.0  # Injection rate (m3/min)
DI = 0.076  # Tubing internal diameter (m)
U = 100.  # Heat transfer coefficient (W/(m2.C))
A = np.pi * DI**2 / 4  # Tubing section area (m2)
VI = QM / (A * 60)  # Velocity (m/s)
DX = 50.  # Element length (m)
N = int(WD / DX)  # Number of elements
DT = 3  # Time step size (s)
TIME = np.zeros(100)  # Time array

# Setting grid depths and temperatures
D = np.zeros(100)
TF = np.zeros(100)
for I in range(N):
    D[I] = DX * I
    TF[I] = TS + GG * D[I]

# Define matrices
P = np.zeros((100, 100))
T = np.zeros((100, 100))
V = np.zeros((100, 100))
SH = np.zeros((100, 100))
DEN = np.zeros((100, 100))
VIS = np.zeros((100, 100))

# Defining the initial conditions
P[0, 0] = PINJ
T[0, 0] = TS
V[0, 0] = VI
_, _, DEN[0, 0] = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
_, VIS[0, 0], _ = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
SH[0 ,0], _, _ = properties(T[0, 0], P[0, 0], "SH", "DEN", "VIS")
# Accumulate viscosity values
#viscosity_values = []

for I in range(N):
    PA = P[0, 0]
    TA = T[0, 0]
    _, _, DEN[I + 1, 0] = properties(TA, PA, "SH", "DEN", "VIS")
    _, VIS[I + 1, 0], _ = properties(TA, PA, "SH", "DEN", "VIS")
    SH[I + 1, 0], _, _ = properties(TA, PA, "SH", "DEN", "VIS")
    DENA = (DEN[I, 0] + DEN[I + 1, 0]) / 2
    VISA = (VIS[I, 0] + VIS[I + 1, 0]) / 2

    # Mass balance equation
    if DEN[I + 1, 0] != 0:
        V[I + 1, 0] = V[I, 0] * DEN[I, 0] / DEN[I + 1, 0]
    else:
        # Handle the case where DEN[I + 1, 0] is zero
        V[I + 1, 0] = 0  # or any other suitable value

    VA = (V[I, 0] + V[I + 1, 0]) / 2

#print("Average Velocity:", VA)
#print("Average Viscosity:", VISA)
#print("Average Density:", DENA)

# Momentun balance equation
for I in range(N):
    GPH = 9.81 * DENA / 1000000.  # Converted from Fortran
    DP = (GPH - fricg(VA, VISA, DI, 0., DENA)) * DX  # Using fricg from Code 2
    
    P[I + 1, 0] = P[I, 0] + DP

# Heat Transfer equation
for I in range(N):
    A1 = A * DEN[I + 1, 0] * V[I + 1, 0] * SH[I + 1, 0] / DX
    B1 = 3.1416 * U * DI
    T[I + 1, 0] = (A1 * T[I, 0] + B1 * TF[I + 1]) / (A1 + B1)

    if abs(P[I + 1, 0] - PA) < 0.1 and abs(T[I + 1, 0] - TA) < 0.1:
        break

    PA = P[I + 1, 0]
    TA = T[I + 1, 0]
   
    #print([I], D[I], P[I, 0], T[I, 0] + 273, TF[I] + 273)  # Print values (remove this line)

# Open files for writing
#with open('Resulpm.txt', 'w') as file1, open('Resultm.txt', 'w') as file2:
    # Write values to Resulpm.txt
#    for I in range(N):
#        file1.write(f'{I} {D[I]} {P[I, 0]} {T[I, 0] + 273} {TF[I] + 273}\n')

    # Write values to Resultm.txt
#    for I in range(N):
#        file2.write(f'{I} {D[I]} {P[I, 0]} {T[I, 0] + 273} {TF[I] + 273}\n')


#Beginning of Time Loop Code

Time = np.zeros(100)  # Time array
Time[0] = 0  # Initialize the first time step



# Setting grid depths and temperatures


for J in range(1, 91):  
    Time[J] = Time[J - 1] + DT / 60. 
    P[0, J] = PINJ
    T[0, J] = TINJ
    V[0, J] = VI
    _, _, DEN[0, J] = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    _, VIS[0, J], _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")
    SH[0, J], _, _ = properties(T[0, J], P[0, J], "SH", "DEN", "VIS")

    for I in range(N):
        PA = P[0, J]  # Note the change here
        TA = T[0, J]  # Note the change here
        _, _, DEN[I + 1, J] = properties(TA, PA, "SH", "DEN", "VIS")
        _, VIS[I + 1, J], _ = properties(TA, PA, "SH", "DEN", "VIS")
        SH[I + 1, J], _, _ = properties(TA, PA, "SH", "DEN", "VIS")
        
        #_, _, DEN[I, J-1] = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        #_, VIS[I, J-1], _ = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        #SH[I, J-1], _, _ = properties(T[I, J-1], P[I, J-1], "SH", "DEN", "VIS")
        
        DENA = (DEN[I, J] + DEN[I + 1, J]) / 2
        VISA = (VIS[I, J] + VIS[I + 1, J]) / 2

        # Mass balance equation
        #if DEN[I + 1, J] != 0:
        V[I + 1, J] = V[I, J] * DEN[I, J] / DEN[I + 1, J]
        #else:
            # Handle the case where DEN[I + 1, J] is zero
            #V[I + 1, J] = 0  # or any other suitable value

        VA = (V[I, J] + V[I + 1, J]) / 2

        #print("Time Step:", Time[J])
        #print("Depth Element:", I)
        #print("Average Velocity:", VA)
        #print("Average Viscosity:", VISA)
        #print("Average Density:", DENA)



        # Momentun balance equation
        GPH = 9.81 * DENA / 1000000.  # Converted from Fortran
        DP = (GPH - fricg(VA, VISA, DI, 0., DENA)) * DX  # Using fricg from Code 2
        P[I + 1, J] = P[I, J] + DP
        #Heat Transfer Equation
        T[I+1, J] = Temp(DEN[I, J-1], DEN[I+1, J-1], DEN[I, J], DEN[I+1, J], V[I, J-1], V[I+1, J-1], V[I, J], V[I+1, J],
                SH[I, J-1], SH[I+1, J-1], SH[I, J], SH[I+1, J], T[I+1, J-1], T[I, J], TF[I+1], U, A, DI, DX, DT)

    
    
    
        if abs(P[I+1, J] - PA) < 0.1 and abs(T[I+1, J] - TA) < 0.1:
            break

            
        #print([I], D[I], P[I, J])

            
        #print([I], D[I], P[I, J])
            
# Writing results to files
#with open("Resulpm2.txt", "w") as f1, open("Resultm2.txt", "w") as f2:
#    for J in range(100):  # Iterate over the entire time range
#        for I in range(N):
#            f1.write(f"{Time[J]} {D[I]} {P[I, J]} ")
#            f2.write(f"{Time[J]} {D[I]} {T[I, J] + 273.} ")
#        f1.write("\n")
#        f2.write("\n")
print("ALL CALCULATIONS COMPLETED")


REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.002981025958006259, GFP: 1.0095960744106522, GFPSI: 0.0228376689608136
REY: 5949540.229004995, GL: 0.0009092364231581488, GT: 1.0095960744106522, F: 0.00298102595800

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F: 0.0030126555989425613, GFP: 1.1143647670086971, GFPSI: 0.025207599648596937
REY: 4263656.576070597, GL: 0.0013857152743877855, GT: 1.1143647670086971, F

KeyboardInterrupt: 